# Bibliotecas

In [0]:
from newsapi import NewsApiClient
from kafka import KafkaProducer
import pickle
import time
from datetime import datetime
import schedule

# Variáveis

In [0]:
# Env
create_folder = "/FileStore/Projeto/data"
apikey = "INSERIR A CHAVE DE API AQUI"


In [0]:
# Cria a pasta de projeto
dbutils.fs.mkdirs(create_folder)

True

# API

In [0]:
class APIClient:
    def __init__(self, key, query) -> None:
        self.__client = NewsApiClient(api_key=key)
        self.__query = query
        self.__all_data = None
        self.__articles = []

    def search(self, page):
        try:
            self.__all_data = self.__client.get_everything(q=self.__query, page = page)
        except:
            self.__all_data = {"status": "end"}


    def get_articles(self):
        page = 1
        self.search(page=page)
        while self.__all_data["status"] == "ok":
            self.__articles += self.__all_data["articles"]
            page += 1
            self.search(page=page)

    @property
    def articles(self):
        return self.__articles
    
    @property
    def all_data(self):
        return self.__all_data

# Critérios de Busca

In [0]:
class Search_Criteria:

    def __init__(self, subject, keywords):
        self.__subject = subject # "genomics"
        self.__keywords = keywords # ["DNA", "genetics", "treatment"]
    
    def add_keyworkd(self, new_keyword):
        self.__keywords.append(new_keyword)

    def remove_keyworkd(self, old_keyword):
        self.__keywords.pop(self.__keywords.index(old_keyword))

    @property
    def subject(self):
        return self.__subject
    
    @subject.setter
    def subject(self, new_subject):
        self.__subject = new_subject

    @property
    def keywords(self):
        return self.__keywords

# Producer

In [0]:
producer_gather = KafkaProducer(bootstrap_servers=["localhost: 9092"])
topic = "genomics-news"

In [0]:
producer_clean = KafkaProducer(bootstrap_servers=["localhost: 9092"])
topic = "clean-data"

# Rotina

In [0]:
# Funçoes de rotina
def update():
    now = datetime.now()

    # Adquirindo todos os artigos
    client.get_articles()

    producer_gather.send(topic, pickle.dumps(client.articles))
    print(f"Producer enviado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    time.sleep(60)

def clean():
    now = datetime.now()

    # Adquirindo todos os artigos
    client.get_articles()

    producer_clean.send(topic, pickle.dumps(client.articles))
    print(f"Producer enviado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    time.sleep(60)

# Execução

In [0]:
# Instanciando pesquisa e palavras chave
criteria = Search_Criteria("genomics", ["DNA", "genetic", "treatment"])

# Instanciando API
client = APIClient(apikey, criteria.subject)

print("Script Iniciado")

schedule.every().hour.at(":12").do(clean)
schedule.every().hour.at(":14").do(update)
schedule.every().hour.at(":16").do(update)
schedule.every().hour.at(":18").do(update)

while True:
    schedule.run_pending()
    time.sleep(10)

Script Iniciado
Producer enviado em 2024-10-21 17:12:02


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can